In [1]:
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd



In [3]:
# Load the cosmetics data from CSV file
cosmetics_df = pd.read_csv("cosmetics.csv")




In [54]:

df = pd.read_csv("cosmetics.csv")

In [60]:
df['Ingredients'][0]


'Algae (Seaweed) Extract, Mineral Oil, Petrolatum, Glycerin, Isohexadecane, Microcrystalline Wax, Lanolin Alcohol, Citrus Aurantifolia (Lime) Extract, Sesamum Indicum (Sesame) Seed Oil, Eucalyptus Globulus (Eucalyptus) Leaf Oil, Sesamum Indicum (Sesame) Seed Powder, Medicago Sativa (Alfalfa) Seed Powder, Helianthus Annuus (Sunflower) Seedcake, Prunus Amygdalus Dulcis (Sweet Almond) Seed Meal, Sodium Gluconate, Copper Gluconate, Calcium Gluconate, Magnesium Gluconate, Zinc Gluconate, Magnesium Sulfate, Paraffin, Tocopheryl Succinate, Niacin, Water, Beta-Carotene, Decyl Oleate, Aluminum Distearate, Octyldodecanol, Citric Acid, Cyanocobalamin, Magnesium Stearate, Panthenol, Limonene, Geraniol, Linalool, Hydroxycitronellal, Citronellol, Benzyl Salicylate, Citral, Sodium Benzoate, Alcohol Denat., Fragrance.'

In [13]:
# Step 1: Prepare Data with Multi-Output Targets
def prepare_data(df):
    """
    Prepares features and targets for multi-output classification.

    Parameters:
    - df (DataFrame): The cosmetics dataset.

    Returns:
    - X (DataFrame): Feature data.
    - y (DataFrame): Multi-output target data for skin types.
    """
    X = df[['Ingredients', 'Price']]
    y = df[['Oily', 'Normal', 'Dry']]
    return X, y



In [15]:
# Step 2: Create Multi-Output Model Pipeline
def create_multioutput_pipeline():
    """
    Creates a multi-output machine learning pipeline with TF-IDF and RandomForestClassifier.

    Returns:
    - pipeline (Pipeline): Scikit-learn pipeline for multi-output classification.
    """
    preprocessor = ColumnTransformer(
        transformers=[
            ('text', TfidfVectorizer(), 'Ingredients'),  # TF-IDF for text features
            ('num', StandardScaler(), ['Price'])         # Standard scaling for price
        ])
    
    # MultiOutputClassifier wraps RandomForest to handle multiple target columns
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
    ])
    return pipeline



In [17]:
# Step 3: Train and Evaluate the Model
def train_model(df):
    """
    Trains a multi-output model for all skin types.

    Parameters:
    - df (DataFrame): The cosmetics dataset.

    Returns:
    - pipeline (Pipeline): Trained model pipeline.
    """
    X, y = prepare_data(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    pipeline = create_multioutput_pipeline()
    pipeline.fit(X_train, y_train)
    
    return pipeline



In [19]:
# Step 4: Generate Recommendations
def generate_recommendations(df, model, skin_type, category, price_range=(0, 100), top_n=5):
    """
    Uses the trained model to predict and rank recommendations based on skin type and price range.

    Parameters:
    - df (DataFrame): The cosmetics dataset.
    - model (Pipeline): Trained multi-output model.
    - skin_type (str): Target skin type to filter ('Oily', 'Normal', 'Dry').
    - category (str): Product category to filter.
    - price_range (tuple): Price range for recommendations, e.g., (min_price, max_price).
    - top_n (int): Number of top recommendations to return.

    Returns:
    - recommendations (DataFrame): DataFrame of top recommendations based on suitability.
    """
    min_price, max_price = price_range
    category_df = df[(df['Label'] == category) & (df['Price'] >= min_price) & (df['Price'] <= max_price)]
    if category_df.empty:
        print(f"No products found for {category} in the price range {price_range}")
        return pd.DataFrame()
    # Predict probabilities for the specified skin type
    probabilities = model.predict_proba(category_df[['Ingredients', 'Price']])
    skin_type_index = {'Oily': 0, 'Normal': 1, 'Dry': 2}[skin_type]
    #suitability_scores = [p[skin_type_index][:, 1] for p in probabilities]  # Probability of suitability for skin type
    # Extract probability of suitability (1st class) for the selected skin type
    skin_type_probabilities = probabilities[0]  # Since this is the 1D array of predicted probabilities

    suitability_scores = skin_type_probabilities[:, 1]  # Get the probabilities for class 1 (suitable)

    # Add suitability scores to the DataFrame
    # Add suitability scores to the DataFrame
    category_df = category_df.assign(predicted_suitability=suitability_scores)
    recommendations = category_df.sort_values(by=['predicted_suitability', 'Rank'], ascending=False).head(top_n)
    
    return recommendations[['Brand', 'Name', 'Price', 'Rank', 'predicted_suitability']]



In [21]:
# Step 5: Train the Model and Get Recommendations
# Train multi-output model for all skin types
multi_skin_model = train_model(cosmetics_df)

# Example usage: Get top 5 recommendations for 'Moisturizer' for 'Oily' skin within a price range
print("\nTop Recommendations for Oily Skin - Moisturizer within Price Range (20, 50):")
recommendations = generate_recommendations(cosmetics_df, multi_skin_model, skin_type='Oily', category='Moisturizer', price_range=(20, 50), top_n=5)
recommendations


Top Recommendations for Oily Skin - Moisturizer within Price Range (20, 50):


,Brand,Name,Price,Rank,predicted_suitability
45,GLOW RECIPE,Watermelon Glow Sleeping Mask,45,4.1,1.00
280,PHILOSOPHY,Hope In A Jar,39,4.0,1.00
36,LANEIGE,Water Sleeping Mask,25,4.4,0.99
46,HERBIVORE,Pink Cloud Rosewater Moisture Crème,48,4.6,0.97
34,IT COSMETICS,Your Skin But Better CC+ Cream Oil-Free Matte ...,38,3.9,0.97


In [22]:

# Example usage: Get top 5 recommendations for 'Moisturizer' for 'Oily' skin within a price range
print("\nTop Recommendations for Oily Skin - Moisturizer within Price Range (20, 50):")
recommendations = generate_recommendations(cosmetics_df, multi_skin_model, skin_type='Dry', category='Cleanser', price_range=(20, 250), top_n=5)
recommendations


Top Recommendations for Oily Skin - Moisturizer within Price Range (20, 50):


,Brand,Name,Price,Rank,predicted_suitability
376,ORIGINS,GinZing™ Refreshing Scrub Cleanser,20,4.5,0.97
328,BOSCIA,Detoxifying Black Charcoal Cleanser,30,4.3,0.97
404,INDIE LEE,Brightening Cleanser,32,4.4,0.96
299,DRUNK ELEPHANT,T.L.C. Framboos™ Glycolic Night Serum,90,4.3,0.96
362,FARMACY,Clean Bee Ultra Gentle Facial Cleanser,28,4.3,0.96


In [16]:
# Example usage: Get top 5 recommendations for 'Moisturizer' for 'Oily' skin within a price range
print("\nTop Recommendations for Oily Skin - Moisturizer within Price Range (20, 50):")
recommendations = generate_recommendations(cosmetics_df, multi_skin_model, skin_type='Oily', category='Cleanser', price_range=(20, 250), top_n=5)
recommendations


Top Recommendations for Oily Skin - Moisturizer within Price Range (20, 50):


,Brand,Name,Price,Rank,predicted_suitability
376,ORIGINS,GinZing™ Refreshing Scrub Cleanser,20,4.5,0.97
328,BOSCIA,Detoxifying Black Charcoal Cleanser,30,4.3,0.97
404,INDIE LEE,Brightening Cleanser,32,4.4,0.96
299,DRUNK ELEPHANT,T.L.C. Framboos™ Glycolic Night Serum,90,4.3,0.96
362,FARMACY,Clean Bee Ultra Gentle Facial Cleanser,28,4.3,0.96


In [17]:


# Save the model
joblib.dump(multi_skin_model, 'recommendation_model.pkl')


['recommendation_model.pkl']

In [27]:
# Load the saved model
import joblib
loaded_model = joblib.load('recommendation_model.pkl')


C:\Users\Dinesh\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Dinesh\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Dinesh\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.5.2 when using version 1.5.1. This might lead to br

In [29]:
# Example usage: Get top 5 recommendations for 'Moisturizer' for 'Oily' skin within a price range
print("\nTop Recommendations for Dry Skin - cleanser within Price Range (20, 150):")
recommendations = generate_recommendations(cosmetics_df, loaded_model, skin_type='Dry', category='Cleanser', price_range=(20, 150), top_n=10)
recommendations


Top Recommendations for Dry Skin - cleanser within Price Range (20, 150):


,Brand,Name,Price,Rank,predicted_suitability
376,ORIGINS,GinZing™ Refreshing Scrub Cleanser,20,4.5,0.9700
328,BOSCIA,Detoxifying Black Charcoal Cleanser,30,4.3,0.9700
404,INDIE LEE,Brightening Cleanser,32,4.4,0.9600
299,DRUNK ELEPHANT,T.L.C. Framboos™ Glycolic Night Serum,90,4.3,0.9600
362,FARMACY,Clean Bee Ultra Gentle Facial Cleanser,28,4.3,0.9600
408,REN CLEAN SKINCARE,Evercalm™ Gentle Cleansing Milk,30,4.3,0.9600
518,KIEHL'S SINCE 1851,Clearly Corrective™ Brightening & Exfoliating ...,29,4.3,0.9600
571,KIEHL'S SINCE 1851,Herbal-Infused Micellar Cleansing Water,28,3.7,0.9600
351,CAUDALIE,Vinopure Natural Salicylic Acid Pore Minimizin...,28,3.8,0.9525
391,IT COSMETICS,Confidence in a Cleanser™ Skin-Transforming Hy...,28,4.7,0.9500


In [31]:
# Example usage: Get top 5 recommendations for 'Moisturizer' for 'Oily' skin within a price range
print("\nTop Recommendations for Normal Skin - cleanser within Price Range (20, 150):")
recommendations = generate_recommendations(cosmetics_df, loaded_model, skin_type='Normal', category='Cleanser', price_range=(20, 150), top_n=10)
recommendations


Top Recommendations for Normal Skin - cleanser within Price Range (20, 150):


,Brand,Name,Price,Rank,predicted_suitability
376,ORIGINS,GinZing™ Refreshing Scrub Cleanser,20,4.5,0.9700
328,BOSCIA,Detoxifying Black Charcoal Cleanser,30,4.3,0.9700
404,INDIE LEE,Brightening Cleanser,32,4.4,0.9600
299,DRUNK ELEPHANT,T.L.C. Framboos™ Glycolic Night Serum,90,4.3,0.9600
362,FARMACY,Clean Bee Ultra Gentle Facial Cleanser,28,4.3,0.9600
408,REN CLEAN SKINCARE,Evercalm™ Gentle Cleansing Milk,30,4.3,0.9600
518,KIEHL'S SINCE 1851,Clearly Corrective™ Brightening & Exfoliating ...,29,4.3,0.9600
571,KIEHL'S SINCE 1851,Herbal-Infused Micellar Cleansing Water,28,3.7,0.9600
351,CAUDALIE,Vinopure Natural Salicylic Acid Pore Minimizin...,28,3.8,0.9525
391,IT COSMETICS,Confidence in a Cleanser™ Skin-Transforming Hy...,28,4.7,0.9500


In [33]:
# Example usage: Get top 5 recommendations for 'Moisturizer' for 'Oily' skin within a price range
print("\nTop Recommendations for Normal Skin - cleanser within Price Range (20, 150):")
recommendations = generate_recommendations(cosmetics_df, loaded_model, skin_type='Normal', category='Cleanser', price_range=(20, 150), top_n=5)
recommendations


Top Recommendations for Normal Skin - cleanser within Price Range (20, 150):


,Brand,Name,Price,Rank,predicted_suitability
376,ORIGINS,GinZing™ Refreshing Scrub Cleanser,20,4.5,0.97
328,BOSCIA,Detoxifying Black Charcoal Cleanser,30,4.3,0.97
404,INDIE LEE,Brightening Cleanser,32,4.4,0.96
299,DRUNK ELEPHANT,T.L.C. Framboos™ Glycolic Night Serum,90,4.3,0.96
362,FARMACY,Clean Bee Ultra Gentle Facial Cleanser,28,4.3,0.96


In [35]:
# Example usage: Get top 5 recommendations for 'Moisturizer' for 'Oily' skin within a price range
print("\nTop Recommendations for Normal Skin - cleanser within Price Range (20, 150):")
recommendations1 = generate_recommendations(cosmetics_df, loaded_model, skin_type='Normal', category='Treatment', price_range=(50, 150), top_n=10)
recommendations1


Top Recommendations for Normal Skin - cleanser within Price Range (20, 150):


,Brand,Name,Price,Rank,predicted_suitability
589,GLAMGLOW,SUPERMUD® Activated Charcoal Treatment,59,4.2,0.98
796,PHILOSOPHY,Time In A Bottle 100% In-Control,76,4.0,0.98
755,DR. DENNIS GROSS SKINCARE,Doctor's Kit Gold Standard Anti-Aging Solution,59,5.0,0.97
667,DR. DENNIS GROSS SKINCARE,Alpha Beta® Medi–Spa Peel,64,4.5,0.97
579,DRUNK ELEPHANT,C-Firma™ Day Serum,80,4.1,0.97
646,CAUDALIE,Vine[Activ] Vitamin C Anti-Wrinkle Serum,52,4.1,0.97
689,DR. DENNIS GROSS SKINCARE,C+ Collagen Brighten & Firm Vitamin C Serum,78,3.9,0.97
621,OLEHENRIKSEN,Glow Cycle Retin-ALT Power Serum,58,4.4,0.96
795,DR. BRANDT SKINCARE,Bright Biotic™ Dark Spot Minimizing Serum,55,4.3,0.96
745,ORIGINS,Dr. Andrew Weil For Origins™ Mega-Mushroom Rel...,56,4.1,0.96


In [37]:
# Example usage: Get top 5 recommendations for 'Moisturizer' for 'Oily' skin within a price range
print("\nTop Recommendations for Normal Skin - cleanser within Price Range (20, 150):")
recommendations = generate_recommendations(cosmetics_df, loaded_model, skin_type='Dry', category='Treatment', price_range=(50, 150), top_n=10)
recommendations


Top Recommendations for Normal Skin - cleanser within Price Range (20, 150):


,Brand,Name,Price,Rank,predicted_suitability
589,GLAMGLOW,SUPERMUD® Activated Charcoal Treatment,59,4.2,0.98
796,PHILOSOPHY,Time In A Bottle 100% In-Control,76,4.0,0.98
755,DR. DENNIS GROSS SKINCARE,Doctor's Kit Gold Standard Anti-Aging Solution,59,5.0,0.97
667,DR. DENNIS GROSS SKINCARE,Alpha Beta® Medi–Spa Peel,64,4.5,0.97
579,DRUNK ELEPHANT,C-Firma™ Day Serum,80,4.1,0.97
646,CAUDALIE,Vine[Activ] Vitamin C Anti-Wrinkle Serum,52,4.1,0.97
689,DR. DENNIS GROSS SKINCARE,C+ Collagen Brighten & Firm Vitamin C Serum,78,3.9,0.97
621,OLEHENRIKSEN,Glow Cycle Retin-ALT Power Serum,58,4.4,0.96
795,DR. BRANDT SKINCARE,Bright Biotic™ Dark Spot Minimizing Serum,55,4.3,0.96
745,ORIGINS,Dr. Andrew Weil For Origins™ Mega-Mushroom Rel...,56,4.1,0.96


In [39]:
recommendations == recommendations1

,Brand,Name,Price,Rank,predicted_suitability
589,True,True,True,True,True
796,True,True,True,True,True
755,True,True,True,True,True
667,True,True,True,True,True
579,True,True,True,True,True
646,True,True,True,True,True
689,True,True,True,True,True
621,True,True,True,True,True
795,True,True,True,True,True
745,True,True,True,True,True


In [41]:

recommendations = generate_recommendations(cosmetics_df, loaded_model, skin_type='Dry', category='Moisturizer', price_range=(151, 150), top_n=10)
recommendations

No products found for Moisturizer in the price range (151, 150)


""


In [44]:
categories = list(cosmetics_df['Label'].unique())

In [46]:
categories

['Moisturizer',
 'Cleanser',
 'Treatment',
 'Face Mask',
 'Eye cream',
 'Sun protect']

In [48]:

recommendations = generate_recommendations(cosmetics_df, loaded_model, skin_type='Dry', category='Face Mask', price_range=(150, 350), top_n=5)
recommendations

,Brand,Name,Price,Rank,predicted_suitability
864,GLAMGLOW,The Ultimate Glow Set,169,0.0,0.940
999,AMOREPACIFIC,TIME RESPONSE Vintage Wash-off Masque,170,5.0,0.930
1080,EVE LOM,WHITE Brightening Mask,160,2.7,0.930
870,SK-II,Brightening Derm Revival Mask,170,4.5,0.900
940,LA MER,Treatment Lotion Hydrating Mask,150,4.1,0.845


In [50]:
import gradio as gr
import pandas as pd
import joblib

# Load the pre-trained model and cosmetics dataset
cosmetics_df = pd.read_csv("cosmetics.csv")
loaded_model = joblib.load("recommendation_model.pkl")

# Define the function for generating recommendations
def recommend_products(skin_type, category, min_price, max_price, top_n):
    """
    Generates product recommendations based on user input.

    Parameters:
    - skin_type (str): Target skin type ('Oily', 'Normal', 'Dry').
    - category (str): Product category ('Moisturizer', 'Cleanser', etc.).
    - min_price (float): Minimum price for filtering products.
    - max_price (float): Maximum price for filtering products.
    - top_n (int): Number of top recommendations to return.

    Returns:
    - recommendations (DataFrame): DataFrame of recommended products.
    """
    price_range = (min_price, max_price)
    recommendations = generate_recommendations(
        cosmetics_df, loaded_model, skin_type, category, price_range, top_n
    )
    if recommendations.empty:
        return f"No products found for {category} in the price range ${min_price} to ${max_price}."
    return recommendations.to_markdown(index=False)

# Gradio interface setup
skin_types = ['Oily', 'Normal', 'Dry']
categories = ['Moisturizer', 'Cleanser', 'Treatment', 'Face Mask', 'Eye cream', 'Sun protect']

with gr.Blocks() as demo:
    gr.Markdown("# Cosmetics Product Recommendation System")
    gr.Markdown("Get the best product recommendations based on your skin type and preferences!")

    with gr.Row():
        skin_type_input = gr.Radio(label="Select Skin Type", choices=skin_types, value='Oily')
        category_input = gr.Dropdown(label="Select Product Category", choices=categories, value='Moisturizer')

    with gr.Row():
        min_price_input = gr.Number(label="Minimum Price", value=20, interactive=True)
        max_price_input = gr.Number(label="Maximum Price", value=50, interactive=True)
        top_n_input = gr.Slider(label="Number of Recommendations", minimum=1, maximum=10, value=5, step=1)

    recommendations_output = gr.Textbox(label="Recommendations", lines=10)

    recommend_button = gr.Button("Get Recommendations")
    recommend_button.click(
        fn=recommend_products,
        inputs=[skin_type_input, category_input, min_price_input, max_price_input, top_n_input],
        outputs=[recommendations_output]
    )

# Launch the Gradio app
demo.launch()


C:\Users\Dinesh\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Dinesh\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Dinesh\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.5.2 when using version 1.5.1. This might lead to br

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [52]:
import gradio as gr
import pandas as pd
import joblib

# Load the pre-trained model and cosmetics dataset
cosmetics_df = pd.read_csv("cosmetics.csv")
loaded_model = joblib.load("recommendation_model.pkl")

# Define the function for generating recommendations
def recommend_products(skin_type, category, min_price, max_price, top_n):
    """
    Generates product recommendations based on user input.

    Parameters:
    - skin_type (str): Target skin type ('Oily', 'Normal', 'Dry').
    - category (str): Product category ('Moisturizer', 'Cleanser', etc.).
    - min_price (float): Minimum price for filtering products.
    - max_price (float): Maximum price for filtering products.
    - top_n (int): Number of top recommendations to return.

    Returns:
    - DataFrame of recommended products.
    """
    price_range = (min_price, max_price)
    recommendations = generate_recommendations(
        cosmetics_df, loaded_model, skin_type, category, price_range, top_n
    )
    if recommendations.empty:
        return pd.DataFrame({"Message": [f"No products found for {category} in the price range ${min_price} to ${max_price}."]})
    return recommendations

# Gradio interface setup
skin_types = ['Oily', 'Normal', 'Dry']
categories = ['Moisturizer', 'Cleanser', 'Treatment', 'Face Mask', 'Eye cream', 'Sun protect']

with gr.Blocks() as demo:
    gr.Markdown("# Cosmetics Product Recommendation System")
    gr.Markdown("Get the best product recommendations based on your skin type and preferences!")

    with gr.Row():
        skin_type_input = gr.Radio(label="Select Skin Type", choices=skin_types, value='Oily')
        category_input = gr.Dropdown(label="Select Product Category", choices=categories, value='Moisturizer')

    with gr.Row():
        min_price_input = gr.Number(label="Minimum Price", value=20, interactive=True)
        max_price_input = gr.Number(label="Maximum Price", value=50, interactive=True)
        top_n_input = gr.Slider(label="Number of Recommendations", minimum=1, maximum=10, value=5, step=1)

    recommendations_output = gr.Dataframe(label="Recommendations")

    recommend_button = gr.Button("Get Recommendations")
    recommend_button.click(
        fn=recommend_products,
        inputs=[skin_type_input, category_input, min_price_input, max_price_input, top_n_input],
        outputs=[recommendations_output]
    )

# Launch the Gradio app
demo.launch()


C:\Users\Dinesh\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Dinesh\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Dinesh\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.5.2 when using version 1.5.1. This might lead to br

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
